In [7]:
##IMPORT SECTION
import pandas
import cx_Oracle as orcl
import getpass
#import re

In [8]:
##CONNECT TO DATABASE USING USER INPUT
def connectToDB():
    uid = input("Enter username (eCommons ID):")
    pwd = getpass.getpass("Enter password:")
    portnum = '51531'
    conn_str = uid + '/' + pwd + '@localhost:' + portnum + "/ORCL"
    conn = orcl.connect(conn_str)
    return conn

conn = connectToDB()

In [19]:
##THE QUERY -- "NOTE:564492681" and "NOTE:465755133" are CONCEPT_CD for 'PTEN Del/Dup' and 'SMC-PTEN'
##WE FIND ALL THE ICD-9 CODES ASSOCIATED WITH EACH PATIENT WHO HAS REPORTED 'PTEN Del/Dup' OR 'SMC-PTEN'
query = 
"""WITH c1 AS ( 
    SELECT DISTINCT patient_num FROM asd_gi.observation_fact WHERE concept_cd IN 
        ('NOTE:564492681', 'NOTE:465755133')
    )
SELECT DISTINCT 
obf.patient_num AS patient, 
obf.concept_cd AS code, 
cd.name_char AS condition
FROM
asd_gi.observation_fact obf,
asd_gi.concept_dimension cd
WHERE
obf.concept_cd = cd.concept_cd AND 
regexp_like(cd.concept_cd, '^ICD9') AND 
obf.patient_num IN (
    SELECT patient_num FROM c1
)"""

In [20]:
#RESULT OF THE SQL QUERY GOES INTO A PANDAS DATA FRAME
res_df = pandas.read_sql(query, conn)

In [43]:
##PERSISTENCE!! THE QUERY TAKES OVER 20 MINUTES TO COMPLETE
res_df.to_csv("/Users/cartik/Desktop/bch-asd.csv")

In [44]:
#CHECK THE DATA FRAME
res_df.head()

,PATIENT,CODE,CONDITION
0,5D17CBC5CBFF6A83E053DC90240ADCBA,ICD9:V21.30,"Low birth weight status, unspecified"
1,5D17CBC62D166A83E053DC90240ADCBA,ICD9:780.02,Transient alteration of awareness
2,5D17CBC637AA6A83E053DC90240ADCBA,ICD9:569.89,Other specified disorders of intestine
3,5D17CBC63DCC6A83E053DC90240ADCBA,ICD9PROC:31.42,Laryngoscopy and other tracheoscopy
4,5D17CBC671316A83E053DC90240ADCBA,ICD9:314.01,Attention deficit disorder of childhood with h...


In [111]:
total_patient_count = len(res_df.groupby('PATIENT'))
total_patient_count

41

In [102]:
##FIND COUNTS OF PATIENTS FOR EACH UNIQUE ICD-9 CODE IN THE DATA FRAME
stats_df = pandas.DataFrame(res_df.groupby(['CODE', 'CONDITION'])['PATIENT'].count())
stats_df.head()

,,PATIENT
CODE,CONDITION,
ICD9:009.0,"Infectious colitis, enteritis, and gastroenteritis",4
ICD9:009.3,Diarrhea of presumed infectious origin,1
ICD9:034.0,Streptococcal sore throat,1
ICD9:038.0,Streptococcal septicemia,1
ICD9:038.19,Other staphylococcal septicemia,1


In [103]:
stats_df = stats_df.rename(index=str, columns={"PATIENT":"PATIENT_COUNT"})

In [112]:
##FIND THE TOP 40 CONDITIONS
top40_conditions = stats_df.nlargest(40, 'PATIENT_COUNT')

In [113]:
top40_conditions

PATIENT_COUNT
CODE        CONDITION                                                        
ICD9:299.00 Autistic disorder, current or active state                     29
            Infantile autism, current or active state                      29
ICD9:315.9  Unspecified delay in development                               25
ICD9:756.0  Anomalies of skull and face bones                              25
            Congenital anomalies of skull and face bones                   25
ICD9:389.9  Unspecified hearing loss                                       17
ICD9:742.4  Other specified congenital anomalies of brain                  14
ICD9:V72.60 Laboratory examination, unspecified                            14
ICD9:315.31 Developmental language disorder                                13
            Expressive language disorder                                   13
ICD9:V82.79 Other genetic screening                                        13
ICD9:564.00 Constipation, unspecified                                      12
ICD9:780.39 Other convulsions                                              12
ICD9:315.39 Other developmental speech disorder                            11
            Other developmental speech or language disorder                11
ICD9:315.8  Other specified delays in development                          11
ICD9:781.3  Lack of coordination                                           10
ICD9:783.3  Feeding difficulties and mismanagement                         10
ICD9:783.40 Lack of normal physiological development, unspe...             10
ICD9:786.2  Cough                                                          10
ICD9:787.03 Vomiting alone                                                 10
ICD9:796.4  Other abnormal clinical findings                               10
ICD9:315.4  Developmental coordination disorder                             9
ICD9:382.9  Unspecified otitis media                                        9
ICD9:530.81 Esophageal reflux                                               9
ICD9:744.89 Other specified congenital anomalies of face an...              9
ICD9:V81.2  Screening for other and unspecified cardiovascu...              9
ICD9:784.60 Symbolic dysfunction, unspecified                               8
ICD9:786.09 Other dyspnea and respiratory abnormality                       8
            Other respiratory abnormalities                                 8
ICD9:787.91 Diarrhea                                                        8
ICD9:V65.3  Dietary surveillance and counseling                             8
ICD9:079.99 Unspecified viral infection                                     7
            Unspecified viral infection in conditions class...              7
ICD9:315.32 Mixed receptive-expressive language disorder                    7
            Receptive language disorder (mixed)                             7
ICD9:327.23 Obstructive sleep apnea (adult) (pediatric)                     7
            Obstructive sleep apnea (adult)(pediatric)                      7
ICD9:381.81 Dysfunction of Eustachian tube                                  7
            Dysfunction of eustachian tube                                  7